In [6]:
import pandas as pd
import numpy as np

total_data = pd.read_excel('merged_data.xlsx', index_col=0)
ksic_data = pd.read_csv('ksic_코드완성.csv', index_col=0)

In [7]:
total_data['시군구'] = total_data['시'].fillna('') + total_data['구'].fillna('')
total_data = total_data.drop(['시', '구', '도로명.1', '도로명_x'], axis=1)
total_data = total_data.rename(columns={'도로명2': '도로명', '도로명_ y' : '도로명주소', '업종이름':'업종소소분류'})
total_data

,상가이름,업종소소분류,업종대분류Cd,업종중분류Cd,업종소분류Cd,업종소소분류Cd,종업원수,매출액등급,번호,도로명_y,읍면동,도로명,시군구
0,대하,가금류 가공 및 저장 처리업,10,1,2,1,8,16,133-2,경북 경산시 압량읍 가일길,압량읍,가일길,경산시
1,주식******,가금류 가공 및 저장 처리업,10,1,2,1,2,1,13-13,경북 경산시 와촌면 하양로,하양읍,하양로,경산시
2,농업*************,가금류 가공 및 저장 처리업,10,1,2,1,3,1,110,경북 경산시 남천면 협석2길,남천면,협석2길,경산시
3,주식*******,가금류 가공 및 저장 처리업,10,1,2,1,7,22,4-13,경북 경산시 박물관로9길,사동,박물관로9길,경산시
4,에스*********,가금류 가공 및 저장 처리업,10,1,2,1,3,1,27,경북 경산시 와촌면 용천길5길,와촌면,용천길5길,경산시
...,...,...,...,...,...,...,...,...,...,...,...,...,...
197676,구두**,그 외 기타 달리 분류되지 않은 개인 서비스업,96,9,9,9,1,1,1,경북 포항시 남구 시청로,대잠동,시청로,포항시남구
197677,육백*****,그 외 기타 달리 분류되지 않은 개인 서비스업,96,9,9,9,3,2,33,경북 포항시 남구 양학천로118번길,대도동,양학천로118번길,포항시남구
197678,헌터********,그 외 기타 달리 분류되지 않은 개인 서비스업,96,9,9,9,3,1,99-8,경북 포항시 북구 용당로,대흥동,용당로,포항시북구
197679,헌터*********,그 외 기타 달리 분류되지 않은 개인 서비스업,96,9,9,9,3,1,4-9,경북 포항시 북구 용당로91번길,대흥동,용당로91번길,포항시북구


In [8]:
new_order = ['상가이름', '시군구', '읍면동', '도로명', '번호', '업종소소분류', '업종대분류Cd', '업종중분류Cd', '업종소분류Cd', '업종소소분류Cd', '종업원수', '매출액등급']
total_data = total_data[new_order]

# 디코딩 작업

In [9]:
ksic_data = ksic_data.drop_duplicates()
# 업종 대분류 데이터프레임
df_대분류 = ksic_data[(ksic_data['업종중분류'] == 0) & (ksic_data['업종소분류'] == 0) & (ksic_data['업종소소분류'] == 0)]

# 업종 중분류 데이터프레임
df_중분류 = ksic_data[(ksic_data['업종대분류'] != 0) & (ksic_data['업종중분류'] != 0) & (ksic_data['업종소분류'] == 0) & (ksic_data['업종소소분류'] == 0)]

# 업종 소분류 데이터프레임
df_소분류 = ksic_data[(ksic_data['업종대분류'] != 0) & (ksic_data['업종중분류'] != 0) & (ksic_data['업종소분류'] != 0) & (ksic_data['업종소소분류'] == 0)]

In [11]:
# 대분류 left join
merged_data = total_data.merge(df_대분류[['업종대분류', 'Industy_name']], 
                                    left_on='업종대분류Cd', 
                                    right_on='업종대분류', 
                                    how='left')

merged_data = merged_data.rename(columns = {'Industy_name' : '업종대분류명'})

# 중분류 left join
merged_data = merged_data.merge(df_중분류[['업종대분류', '업종중분류', 'Industy_name']], 
                                     left_on=['업종대분류Cd', '업종중분류Cd'], 
                                     right_on=['업종대분류', '업종중분류'], 
                                     how = 'left')

merged_data = merged_data.rename(columns = {'Industy_name' : '업종중분류명'})

# 소분류 left join
merged_data = merged_data. merge(df_소분류[['업종대분류', '업종중분류', '업종소분류', 'Industy_name']],
                                      left_on=['업종대분류Cd', '업종중분류Cd', '업종소분류Cd'],
                                      right_on=['업종대분류', '업종중분류', '업종소분류'],
                                      how = 'left')
merged_data = merged_data.rename(columns = {'Industy_name' : '업종소분류명', '업종이름' : '업종소소분류'})

merged_data = merged_data.drop(['업종대분류_y', '업종대분류_x', '업종중분류_x', '업종중분류_y', '업종대분류', '업종소분류'], axis=1)

merged_data

,상가이름,시군구,읍면동,도로명,번호,업종소소분류,업종대분류Cd,업종중분류Cd,업종소분류Cd,업종소소분류Cd,종업원수,매출액등급,업종대분류명,업종중분류명,업종소분류명
0,대하,경산시,압량읍,가일길,133-2,가금류 가공 및 저장 처리업,10,1,2,1,8,16,식료품 제조업,"도축, 육류 가공 및 저장 처리업",육류 가공 및 저장 처리업
1,주식******,경산시,하양읍,하양로,13-13,가금류 가공 및 저장 처리업,10,1,2,1,2,1,식료품 제조업,"도축, 육류 가공 및 저장 처리업",육류 가공 및 저장 처리업
2,농업*************,경산시,남천면,협석2길,110,가금류 가공 및 저장 처리업,10,1,2,1,3,1,식료품 제조업,"도축, 육류 가공 및 저장 처리업",육류 가공 및 저장 처리업
3,주식*******,경산시,사동,박물관로9길,4-13,가금류 가공 및 저장 처리업,10,1,2,1,7,22,식료품 제조업,"도축, 육류 가공 및 저장 처리업",육류 가공 및 저장 처리업
4,에스*********,경산시,와촌면,용천길5길,27,가금류 가공 및 저장 처리업,10,1,2,1,3,1,식료품 제조업,"도축, 육류 가공 및 저장 처리업",육류 가공 및 저장 처리업
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197726,구두**,포항시남구,대잠동,시청로,1,그 외 기타 달리 분류되지 않은 개인 서비스업,96,9,9,9,1,1,기타 개인 서비스업,그 외 기타 개인 서비스업,그 외 기타 분류안된 개인 서비스업
197727,육백*****,포항시남구,대도동,양학천로118번길,33,그 외 기타 달리 분류되지 않은 개인 서비스업,96,9,9,9,3,2,기타 개인 서비스업,그 외 기타 개인 서비스업,그 외 기타 분류안된 개인 서비스업
197728,헌터********,포항시북구,대흥동,용당로,99-8,그 외 기타 달리 분류되지 않은 개인 서비스업,96,9,9,9,3,1,기타 개인 서비스업,그 외 기타 개인 서비스업,그 외 기타 분류안된 개인 서비스업
197729,헌터*********,포항시북구,대흥동,용당로91번길,4-9,그 외 기타 달리 분류되지 않은 개인 서비스업,96,9,9,9,3,1,기타 개인 서비스업,그 외 기타 개인 서비스업,그 외 기타 분류안된 개인 서비스업


In [39]:
new_order = ['상가이름', '시군구', '읍면동', '도로명', '번호',
             '업종대분류Cd', '업종중분류Cd', '업종소분류Cd', '업종소소분류Cd', '종업원수', '매출액등급',
             '업종대분류명', '업종소소분류']
merged_data = merged_data[new_order]
merged_data
merged_data = merged_data.drop_duplicates()


In [41]:
merged_data.to_csv('merged_data_완성본.csv')